In [ ]:
#START FROM HERE

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer

# Load the CSV file into a Pandas dataframe
df = pd.read_csv('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/Train.csv')

In [ ]:
#preprocess
df

,Text,Emotion
0,I'm so cross I got a poor grade on the quiz,Anger
1,I'm so upset I couldn't do better on the exam,Anger
2,I'm so mad I didn't get the marks I was hoping...,Anger
3,I'm so furious I couldn't ace the test,Anger
4,I'm so irate I got a lower grade than I wanted,Anger
...,...,...
1180,I'm elated that I passed the exam and can now ...,happy
1181,I'm disheartened that I didn't get the score I...,sad
1182,I'm scared that I won't be able to finish the ...,fear
1183,I'm stunned that I got a higher score on the e...,surprise


In [ ]:

# Initialize the stemmer
stemmer = PorterStemmer()

# Define a function to stem a given string
def stem_text(text):
    words = nltk.word_tokenize(text)
    return ' '.join([stemmer.stem(w) for w in words])

# Loop over the 'emotion' column and apply stemming to each string
for i in range(len(df)):
    text = str(df['Emotion'][i])
    df.at[i, 'Emotion'] = stem_text(text)

# df = df.drop('label', axis = 1)
# df


In [ ]:
print(df['Emotion'].unique())

['anger' 'surpris' 'sad' 'happi' 'nan' 'fear' 'sad .' 'anfer']


In [ ]:
# replace "anfer" with "anger" and "sad ." with "sad"
df['Emotion'] = df['Emotion'].replace({'anfer': 'anger', 'sad .': 'sad'})

# print the updated DataFrame
print(df['Emotion'].unique())

['anger' 'surpris' 'sad' 'happi' 'nan' 'fear']


In [ ]:
# Create a dictionary to map emotions to labels
emotion_labels = {'happi': 1, 'sad': 2, 'anger': 3, 'fear': 4, 'surpris': 0}

# Map the emotions to labels
df['label'] = df['Emotion'].map(emotion_labels)

# Drop rows with invalid emotions
valid_emotions = list(emotion_labels.keys())
df = df[df['Emotion'].isin(valid_emotions)]

# Drop the 'emotion' column
df = df.drop('Emotion', axis=1)

# Print the resulting dataframe
df


,Text,label
0,I'm so cross I got a poor grade on the quiz,3.0
1,I'm so upset I couldn't do better on the exam,3.0
2,I'm so mad I didn't get the marks I was hoping...,3.0
3,I'm so furious I couldn't ace the test,3.0
4,I'm so irate I got a lower grade than I wanted,3.0
...,...,...
1180,I'm elated that I passed the exam and can now ...,1.0
1181,I'm disheartened that I didn't get the score I...,2.0
1182,I'm scared that I won't be able to finish the ...,4.0
1183,I'm stunned that I got a higher score on the e...,0.0


In [ ]:

df['label'] = df['label'].astype(int)
label_counts = df['label'].value_counts()

print(label_counts)


1    328
2    240
0    209
3    186
4    179
Name: label, dtype: int64


In [ ]:
# Define the file path and name
csv_file_path = '/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/preprocessed_Train.csv'  # replace with the actual file path and file name
df.to_csv(csv_file_path, index=False)

# Export the DataFrame to a CSV file
print(f"The DataFrame has been exported to {csv_file_path}")

The DataFrame has been exported to /content/gdrive/MyDrive/EXAM EMOTIONS MODEL/preprocessed_Train.csv


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.6 MB/s eta 0:00:00


In [ ]:
# Load the CSV file into a Pandas dataframe
df = pd.read_csv('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/preprocessed_Train.csv')

df = df.loc[df['Text'].apply(lambda x: isinstance(x, str))]

# print the updated DataFrame
print(df)

                                                   Text  label
0           I'm so cross I got a poor grade on the quiz      3
1         I'm so upset I couldn't do better on the exam      3
2     I'm so mad I didn't get the marks I was hoping...      3
3                I'm so furious I couldn't ace the test      3
4        I'm so irate I got a lower grade than I wanted      3
...                                                 ...    ...
1137  I'm elated that I passed the exam and can now ...      1
1138  I'm disheartened that I didn't get the score I...      2
1139  I'm scared that I won't be able to finish the ...      4
1140  I'm stunned that I got a higher score on the e...      0
1141  I'm irritated that the exam questions didn't c...      3

[1139 rows x 2 columns]


In [ ]:
import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments


# Split the dataset into train, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Load the tokenizer and encode the text data
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
train_encodings = tokenizer(train_df['Text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['Text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['Text'].tolist(), truncation=True, padding=True)

# Convert the label data to PyTorch tensors
train_labels = torch.tensor(train_df['label'].tolist())
val_labels = torch.tensor(val_df['label'].tolist())
test_labels = torch.tensor(test_df['label'].tolist())




In [ ]:

# Create the model
model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=5)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=50,  # <-- change eval_steps
    save_steps=50,  # <-- add save_steps parameter
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False
)



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classi

In [ ]:
unique_values_Label = df['Label'].unique()
print(unique_values_Label)

In [ ]:
# Define the Trainer object
trainer = Trainer(
    model=model,                                                    # the instantiated 🤗 Transformers model to be trained
    args=training_args,                                             # training arguments, defined above
    train_dataset=torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), 
                                               torch.tensor(train_encodings['attention_mask']), 
                                               torch.tensor(train_labels)),
    eval_dataset=torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']), 
                                               torch.tensor(val_encodings['attention_mask']), 
                                               torch.tensor(val_labels)),
    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]), 
                                'attention_mask': torch.stack([f[1] for f in data]), 
                                'labels': torch.tensor([f[2] for f in data])},
    # compute_metrics=lambda preds, labels: {'accuracy': (preds.argmax(axis=1) == labels).mean()}
)


<ipython-input-55-73903251dbeb>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels)),
<ipython-input-55-73903251dbeb>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(val_labels)),


In [ ]:
# Fine-tune the model
flag = False

if torch.cuda.is_available():
  trainer.train()
  flag= True


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
50,1.599300,1.585793
100,1.452600,1.240462
150,0.089300,0.108022
200,0.042800,0.155961
250,0.043900,0.082721
300,0.028100,0.059135
350,0.012000,0.064510
400,0.081600,0.286876
450,0.025500,0.132257
500,0.086800,0.152975


In [ ]:
if flag == True:
  model.save_pretrained('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/myModel_better')

In [ ]:
if flag == False:
  model = RobertaForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/myModel_better')


In [ ]:
def predict_emotion_label(text: str) -> int:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_encoding = tokenizer.encode_plus(text,
                                            truncation=True,
                                            padding=True,
                                            return_tensors='pt').to(device)
    model.to(device)
    outputs = model(input_ids=input_encoding['input_ids'],
                    attention_mask=input_encoding['attention_mask']).logits.to('cpu')
    predicted_label = torch.argmax(outputs, axis=1).item()
    return predicted_label


In [ ]:
predict_emotion_label("I'm so frustrated with how poorly I did on this exam")

3

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/Test.csv')

In [ ]:
df

,text,emotion,label
0,I studied all night for my exam and I still do...,fear,4
1,I'm relieved that my exams are finally over.,happy,1
2,I'm disappointed that I didn't score higher on...,sad,2
3,I'm anxious about my final exam next week.,fear,4
4,I'm proud of myself for passing my chemistry e...,happy,1
...,...,...,...
379,I'm feeling happy that I was able to pass my F...,happy,1
380,I'm thrilled that I got accepted into the grad...,happy,1
381,I'm so glad that I took the time to study for ...,happy,1
382,I'm beaming with joy after finding out that I ...,happy,1


In [ ]:

# Initialize the stemmer
stemmer = PorterStemmer()

# Define a function to stem a given string
def stem_text(text):
    words = nltk.word_tokenize(text)
    return ' '.join([stemmer.stem(w) for w in words])

# Loop over the 'emotion' column and apply stemming to each string
for i in range(len(df)):
    text = str(df['emotion'][i])
    df.at[i, 'emotion'] = stem_text(text)

# df = df.drop('intensity', axis = 1)
# df

print(df['emotion'].unique())
# Create a dictionary to map emotions to labels
emotion_labels = {'happi': 1, 'sad': 2, 'anger': 3, 'fear': 4, 'surpris': 0}

# Map the emotions to labels
df['label'] = df['emotion'].map(emotion_labels)

# Drop rows with invalid emotions
valid_emotions = list(emotion_labels.keys())
df = df[df['emotion'].isin(valid_emotions)]

# Drop the 'emotion' column
df = df.drop('emotion', axis=1)

# Print the resulting dataframe
df

df['label'] = df['label'].astype(int)
label_counts = df['label'].value_counts()

print(label_counts)
from google.colab import drive
import pandas as pd


# Define the file path and name
file_path = '/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/Test_data.csv'


# Export the DataFrame to a CSV file
df.to_csv(file_path, index=False)



['fear' 'happi' 'sad' 'anger' 'surpris']
4    90
1    88
3    86
2    79
0    41
Name: label, dtype: int64


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/Test_data.csv')

In [ ]:
def predict_emotion(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_encoding = tokenizer.encode_plus(text,
                                            truncation=True,
                                            padding=True,
                                            return_tensors='pt').to(device)
    model.to(device)
    outputs = model(input_ids=input_encoding['input_ids'],
                    attention_mask=input_encoding['attention_mask']).logits.to('cpu')
    predicted_label = torch.argmax(outputs, axis=1).item()
    return predicted_label


# Apply the predict_emotion function to each row of the dataframe
predicted_labels = df['text'].apply(predict_emotion)

# Calculate the accuracy by comparing the predicted labels to the actual labels
accuracy = (predicted_labels == df['label']).mean()
print(f"Accuracy: {accuracy}")


Accuracy: 0.9791666666666666


In [ ]:
#detect emotion 
def predict_emotion(text: str) -> int:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_encoding = tokenizer.encode_plus(text,
                                            truncation=True,
                                            padding=True,
                                            return_tensors='pt').to(device)
    model.to(device)
    outputs = model(input_ids=input_encoding['input_ids'],
                    attention_mask=input_encoding['attention_mask']).logits.to('cpu')
    predicted_label = torch.argmax(outputs, axis=1).item()
    
    # Map the predicted label to the actual emotion
    emotion_map = {0'surprise',1:'happy', 2:'sad', 3:'anger', 4:'fear'}
    predicted_emotion = emotion_map[predicted_label]
    
    return predicted_emotion


In [ ]:
predict_emotion("How did I get an O grade in this subject, i hadn't even studied well!")

'surprise'

In [ ]:
# if flag == True:
#   def predict_emotion_label(text: str) -> int:
#         input_encoding = tokenizer.encode_plus(text,
#                                           truncation=True,
#                                           padding=True,
#                                           return_tensors='pt').to('cuda')

#         # Move the input tensors to CPU device
#         input_ids = input_encoding['input_ids'].cpu()
#         attention_mask = input_encoding['attention_mask'].cpu()

#         # Make the prediction
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits.to('cpu')
#         predicted_label = torch.argmax(outputs, axis=1).item()


#         return predicted_label


In [ ]:
#prev traing parameters# # Create the model

# model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=5)

# # Define the training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs= 20,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     evaluation_strategy='steps',
#     eval_steps=500,
#     save_total_limit=1,
#     load_best_model_at_end=True,
#     metric_for_best_model='eval_loss',
#     greater_is_better=False
# )
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=20,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     evaluation_strategy='steps',
#     eval_steps=800,  # <-- change eval_steps to 500
#     save_steps=1600,  # <-- add save_steps parameter
#     save_total_limit=1,
#     load_best_model_at_end=True,
#     metric_for_best_model='eval_loss',
#     greater_is_better=False
# )